In [15]:
#Import dependencies
import pandas as pd
import requests
import json
from config import cost_api_key
import os
import html5lib as html
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from splinter import Browser 
from bs4 import BeautifulSoup as bs


In [17]:
#Setup splinter parameters
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
#browser.implicit_wait(30)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\12254\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [18]:
#Scrape URL for metro area statistics. Will be merged/cleaned and used later for census data. 
###Note: This table is not used below in the API call.
url = 'https://en.wikipedia.org/wiki/List_of_metropolitan_statistical_areas'
browser.visit(url)
html = browser.html
html_df = pd.read_html(html)

#Print for verification
html_df[1]

,Rank,Metropolitan statistical area,2020 Census,2010 Census,% change,Encompassing combined statistical area
0,1,"New York-Newark-Jersey City, NY-NJ-PA MSA",20140470,18897109,+6.58%,"New York-Newark, NY-NJ-CT-PA CSA"
1,2,"Los Angeles-Long Beach-Anaheim, CA MSA",13200998,12828837,+2.90%,"Los Angeles-Long Beach, CA CSA"
2,3,"Chicago-Naperville-Elgin, IL-IN-WI MSA",9618502,9461105,+1.66%,"Chicago-Naperville, IL-IN-WI CSA"
3,4,"Dallas-Fort Worth-Arlington, TX MSA",7637387,6366542,+19.96%,"Dallas-Fort Worth, TX-OK CSA"
4,5,"Houston-The Woodlands-Sugar Land, TX MSA",7122240,5920416,+20.30%,"Houston-The Woodlands, TX CSA"
...,...,...,...,...,...,...
379,380,"Danville, IL MSA",74188,81625,−9.11%,NaN
380,381,"Lewiston, ID-WA MSA",64375,60888,+5.73%,NaN
381,382,"Enid, OK MSA",62846,60580,+3.74%,NaN
382,383,"Walla Walla, WA MSA",62584,58781,+6.47%,"Kennewick-Richland-Walla Walla, WA CSA"


In [19]:
browser.quit()

In [81]:
#Store to csv file
html_df_final = pd.DataFrame(html_df[1])
#html_df_final
html_df_final.to_csv('../Resources/metro_census_data.csv')


In [20]:
#Read csv with unique Fortune500 cities.
###Note: This is used in the API call below. 
file = "../Resources/unique_cities.csv"
df = pd.read_csv(file)
df.head()

,Unnamed: 0,city,state
0,0,Bentonville,AR
1,1,Omaha,NE
2,2,Cupertino,CA
3,3,Irving,TX
4,4,San Francisco,CA


In [27]:
#Initialize empty data list
data = []

#Convert city column from csv to a list
city_list = df['city'].to_list()
city_list
#print(city_list, len(city_list))


['Bentonville', 'Omaha', 'Cupertino', 'Irving', 'San Francisco', 'Minnetonka', 'Woonsocket', 'Detroit', 'Dallas', 'Dearborn', 'Chesterbrook', 'Seattle', 'Fairfield', 'New York', 'Dublin', 'Issaquah', 'Deerfield', 'Cincinnati', 'San Ramon', 'Washington', 'Saint Louis', 'Atlanta', 'Chicago', 'Charlotte', 'Mountain View', 'Redmond', 'Indianapolis', 'Philadelphia', 'Armonk', 'Bloomington', 'Houston', 'New Brunswick', 'San Antonio', 'Minneapolis', 'Mclean', 'Mooresville', 'Round Rock', 'Hartford', 'Purchase', 'Decatur', 'Santa Clara', 'Newark', 'Boise', 'Farmington', 'Findlay', 'Burbank', 'Louisville', 'Bethesda', 'Memphis', 'Palo Alto', 'San Jose', 'Midland', 'Nashville', 'Fort Worth', 'Columbus', 'Kenilworth', 'Bloomfield', 'Richfield', 'Morris Township', 'Peoria', 'Boston', 'Springfield', 'Redwood City', 'Springdale', 'Northbrook', 'Lakeland', 'Framingham', 'Beaverton', 'Falls Church', 'Camp Hill', 'Foster City', 'Inver Grove Height', 'Maplewood', 'Stamford', 'Milwaukee', 'Menlo Park', '

In [28]:
#Loop through city_list to create dictionary for API call
for city in city_list:
    data.append({"name":city,"country":"United States"})

In [29]:
#Print data list for verification
data

[{'name': 'Bentonville', 'country': 'United States'},
 {'name': 'Omaha', 'country': 'United States'},
 {'name': 'Cupertino', 'country': 'United States'},
 {'name': 'Irving', 'country': 'United States'},
 {'name': 'San Francisco', 'country': 'United States'},
 {'name': 'Minnetonka', 'country': 'United States'},
 {'name': 'Woonsocket', 'country': 'United States'},
 {'name': 'Detroit', 'country': 'United States'},
 {'name': 'Dallas', 'country': 'United States'},
 {'name': 'Dearborn', 'country': 'United States'},
 {'name': 'Chesterbrook', 'country': 'United States'},
 {'name': 'Seattle', 'country': 'United States'},
 {'name': 'Fairfield', 'country': 'United States'},
 {'name': 'New York', 'country': 'United States'},
 {'name': 'Dublin', 'country': 'United States'},
 {'name': 'Issaquah', 'country': 'United States'},
 {'name': 'Deerfield', 'country': 'United States'},
 {'name': 'Cincinnati', 'country': 'United States'},
 {'name': 'San Ramon', 'country': 'United States'},
 {'name': 'Washingto

In [23]:
#Convert python list object into json object
payload_json = json.dumps(data)
print(payload_json)

[{"name": "Bentonville", "country": "United States"}, {"name": "Omaha", "country": "United States"}, {"name": "Cupertino", "country": "United States"}, {"name": "Irving", "country": "United States"}, {"name": "San Francisco", "country": "United States"}, {"name": "Minnetonka", "country": "United States"}, {"name": "Woonsocket", "country": "United States"}, {"name": "Detroit", "country": "United States"}, {"name": "Dallas", "country": "United States"}, {"name": "Dearborn", "country": "United States"}, {"name": "Chesterbrook", "country": "United States"}, {"name": "Seattle", "country": "United States"}, {"name": "Fairfield", "country": "United States"}, {"name": "New York", "country": "United States"}, {"name": "Dublin", "country": "United States"}, {"name": "Issaquah", "country": "United States"}, {"name": "Deerfield", "country": "United States"}, {"name": "Cincinnati", "country": "United States"}, {"name": "San Ramon", "country": "United States"}, {"name": "Washington", "country": "Uni

In [34]:
#Setup API parameters and request. Payload includes cities and currency.
###Note: Will need to hide cost_api_key before turning in.
api_url = "https://cities-cost-of-living1.p.rapidapi.com/get_cities_details"
currency = f'["USD"]'
payload = {"cities":payload_json, "currencies":currency}
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'x-rapidapi-host': "cities-cost-of-living1.p.rapidapi.com",
    'x-rapidapi-key': cost_api_key
    }
try:
    response = requests.request("POST", api_url, data=payload, headers=headers)
    print(response.text)
except:
    print(f'API call failed.')



{"data":[{"cost_of_living_details":[{"currency":"USD","details":[{"Item":"Estimated Monthly Costs Without Rent","Range":"N/A","Value":"977.17"},{"Item":"Meal, Inexpensive Restaurant","Range":"9.98-29.94","Value":"15.48"},{"Item":"Meal for 2 People, Mid-range Restaurant, Three-course","Range":"39.92-89.83","Value":"59.89"},{"Item":"McMeal at McDonalds (or Equivalent Combo Meal)","Range":"6.98-8.99","Value":"7.99"},{"Item":"Domestic Beer (1 pint draught)","Range":"3.0-5.99","Value":"3.99"},{"Item":"Imported Beer (12 oz small bottle)","Range":"4.99-7.99","Value":"6.49"},{"Item":"Cappuccino (regular)","Range":"3.0-5.99","Value":"4.37"},{"Item":"Coke/Pepsi (12 oz small bottle)","Range":"1.7-3.0","Value":"2.07"},{"Item":"Water (12 oz small bottle)","Range":"1.5-1.99","Value":"1.71"},{"Item":"Milk (regular), (1 gallon)","Range":"1.69-3.99","Value":"2.59"},{"Item":"Loaf of Fresh White Bread (1 lb)","Range":"1.5-4.99","Value":"2.25"},{"Item":"Rice (white), (1 lb)","Range":"1.0-3.99","Value":"2.

In [53]:
#Convert response into json format
response_json = response.json()
response_json


{'data': [{'cost_of_living_details': [{'currency': 'USD',
     'details': [{'Item': 'Estimated Monthly Costs Without Rent',
       'Range': 'N/A',
       'Value': '977.17'},
      {'Item': 'Meal, Inexpensive Restaurant',
       'Range': '9.98-29.94',
       'Value': '15.48'},
      {'Item': 'Meal for 2 People, Mid-range Restaurant, Three-course',
       'Range': '39.92-89.83',
       'Value': '59.89'},
      {'Item': 'McMeal at McDonalds (or Equivalent Combo Meal)',
       'Range': '6.98-8.99',
       'Value': '7.99'},
      {'Item': 'Domestic Beer (1 pint draught)',
       'Range': '3.0-5.99',
       'Value': '3.99'},
      {'Item': 'Imported Beer (12 oz small bottle)',
       'Range': '4.99-7.99',
       'Value': '6.49'},
      {'Item': 'Cappuccino (regular)', 'Range': '3.0-5.99', 'Value': '4.37'},
      {'Item': 'Coke/Pepsi (12 oz small bottle)',
       'Range': '1.7-3.0',
       'Value': '2.07'},
      {'Item': 'Water (12 oz small bottle)',
       'Range': '1.5-1.99',
       'Value

In [62]:
response_length = len(response_json["data"])
response_length

64

In [73]:
#Create dictionary based upon API response; will be imported into mongodb
all_cities_data = []


i = 0
while i < response_length:
    city = response_json["data"][i]["name"]
    state = response_json["data"][i]["us_state"]
    cost_of_living_index = response_json["data"][i]["cost_of_living_index"]
    purchasing_power = response_json["data"][i]["local_purchasing_power_index"]
    groceries_index = response_json["data"][i]["groceries_index"]
    rent_index = response_json["data"][i]["rent_index"]
    restaurant_price_index = response_json["data"][i]["restaurant_price_index"]
    

    all_cities_data.append({"City": city,
                            "State": state,
                                "Cost of Living Index": cost_of_living_index,
                                "Local Purchasing Power": purchasing_power,
                                "Groceries Index": groceries_index,
                                "Rent Index": rent_index,
                                "Restaurant Price Index": restaurant_price_index})

    i += 1

In [74]:
all_cities_data

[{'City': 'Omaha',
  'State': 'NE',
  'Cost of Living Index': '72.72',
  'Local Purchasing Power': '102.23',
  'Groceries Index': '71.12',
  'Rent Index': '30.41',
  'Restaurant Price Index': '71.33'},
 {'City': 'San Francisco',
  'State': 'CA',
  'Cost of Living Index': '94.21',
  'Local Purchasing Power': '117.06',
  'Groceries Index': '97.72',
  'Rent Index': '116.83',
  'Restaurant Price Index': '93.57'},
 {'City': 'Detroit',
  'State': 'MI',
  'Cost of Living Index': '68.57',
  'Local Purchasing Power': '102.08',
  'Groceries Index': '65.19',
  'Rent Index': '40.02',
  'Restaurant Price Index': '67.95'},
 {'City': 'Dallas',
  'State': 'TX',
  'Cost of Living Index': '66.62',
  'Local Purchasing Power': '154.38',
  'Groceries Index': '60.73',
  'Rent Index': '50.19',
  'Restaurant Price Index': '72.03'},
 {'City': 'Seattle',
  'State': 'WA',
  'Cost of Living Index': '86.25',
  'Local Purchasing Power': '128.62',
  'Groceries Index': '84.31',
  'Rent Index': '68.46',
  'Restaurant 

In [75]:
#Convert to dataframe  
cost_of_living_df = pd.DataFrame(all_cities_data)
cost_of_living_df

,City,State,Cost of Living Index,Local Purchasing Power,Groceries Index,Rent Index,Restaurant Price Index
0,Omaha,NE,72.72,102.23,71.12,30.41,71.33
1,San Francisco,CA,94.21,117.06,97.72,116.83,93.57
2,Detroit,MI,68.57,102.08,65.19,40.02,67.95
3,Dallas,TX,66.62,154.38,60.73,50.19,72.03
4,Seattle,WA,86.25,128.62,84.31,68.46,87.80
...,...,...,...,...,...,...,...
59,Little Rock,AR,59.39,119.10,56.72,26.66,65.06
60,Birmingham,AL,74.43,83.41,76.01,30.25,72.17
61,Oakland,CA,90.52,97.49,97.29,86.96,75.35
62,Buffalo,NY,73.38,118.84,70.73,29.31,71.10
